In [ ]:
!pip install -r requirements.txt

In [11]:
!pip install sentence-transformers

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
Using cached sentence_transformers-3.3.1-py3-none-any.whl (268 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)


In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
import os
import pandas as pd
import re
from sentence_transformers import SentenceTransformer

C:\Users\52477\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Setting the enviroment
DATA_PATH = r"data"
CHROMA_PATH = r"chroma_db"

chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = chroma_client.get_or_create_collection(name = "courses")
files = pd.read_csv("InterestingLinks.csv")
model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
def remove_emojis(text):
    # Expresión regular que cubre la mayoría de los emojis
    emoji_pattern = re.compile(
        "[\U0001F600-\U0001F64F"      # Emoticonos
        "\U0001F300-\U0001F5FF"      # Símbolos y pictogramas
        "\U0001F680-\U0001F6FF"      # Transporte y mapas
        "\U0001F700-\U0001F77F"      # Alquimia y símbolos
        "\U0001F780-\U0001F7FF"      # Geometría y símbolos varios
        "\U0001F800-\U0001F8FF"      # Suplemento de flechas
        "\U0001F900-\U0001F9FF"      # Emoticonos adicionales
        "\U0001FA00-\U0001FA6F"      # Emojis varios
        "\U0001FA70-\U0001FAFF"      # Más emojis
        "\U00002702-\U000027B0"      # Caracteres varios (usados para simbolos de puntuación, etc.)
        "]+", flags=re.UNICODE)
    
    # Reemplazar los emojis encontrados por una cadena vacía
    return re.sub(emoji_pattern, '', text)

# Función para leer todos los archivos .txt y eliminar los emojis
# Función para leer, limpiar y escribir los archivos .txt
def clean_and_write_txt_files(directory_path, write_to_new_file=False):
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            
            # Leer el archivo de texto
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                
                # Eliminar los emojis del texto
                cleaned_text = remove_emojis(text)
                
                if write_to_new_file:
                    # Si se desea escribir a un nuevo archivo, crear un nombre único
                    new_file_path = os.path.join(directory_path, f"new_{filename}")
                    with open(new_file_path, 'w', encoding='utf-8') as new_file:
                        new_file.write(cleaned_text)
                    print(f"Archivo limpio guardado como: {new_file_path}")
                else:
                    # Si se desea sobrescribir el archivo original
                    with open(file_path, 'w', encoding='utf-8') as file:
                        file.write(cleaned_text)
                    print(f"Archivo original sobrescrito: {file_path}")

# Directorio donde están tus archivos .txt
directory_path = 'data'

# Llamar a la función para limpiar y reescribir los archivos
# Si `write_to_new_file` es True, los archivos limpios se guardarán con un prefijo "cleaned_"
# Si es False, se sobrescribirán los archivos originales.
clean_and_write_txt_files(directory_path, write_to_new_file=False)

Archivo original sobrescrito: data\0.txt
Archivo original sobrescrito: data\1.txt
Archivo original sobrescrito: data\10.txt
Archivo original sobrescrito: data\11.txt
Archivo original sobrescrito: data\12.txt
Archivo original sobrescrito: data\13.txt
Archivo original sobrescrito: data\14.txt
Archivo original sobrescrito: data\15.txt
Archivo original sobrescrito: data\16.txt
Archivo original sobrescrito: data\17.txt
Archivo original sobrescrito: data\18.txt
Archivo original sobrescrito: data\19.txt
Archivo original sobrescrito: data\2.txt
Archivo original sobrescrito: data\20.txt
Archivo original sobrescrito: data\21.txt
Archivo original sobrescrito: data\22.txt
Archivo original sobrescrito: data\3.txt
Archivo original sobrescrito: data\4.txt
Archivo original sobrescrito: data\5.txt
Archivo original sobrescrito: data\6.txt
Archivo original sobrescrito: data\7.txt
Archivo original sobrescrito: data\8.txt
Archivo original sobrescrito: data\9.txt


In [4]:
# Cargar los archivos de texto
documents = []
metadata = []
ids = []
j = 0
#Loading the document
for filename in os.listdir(DATA_PATH):
    if filename.endswith(".txt"):
            try:
                file_path = os.path.join(DATA_PATH, filename)
                loader = TextLoader(file_path)  # Usamos TextLoader para cargar el archivo de texto
                raw_documents = loader.load()

                # Dividir el documento en trozos (chunks)
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=300,
                    chunk_overlap=100,
                    length_function=len,
                    is_separator_regex=False,
                )

                chunks = text_splitter.split_documents(raw_documents)  # Preparar los chunks para ser agregados a Chroma

                # Procesar cada chunk para agregarlo a Chroma
                for i, chunk in enumerate(chunks):
                    documents.append(chunk.page_content)
                    embedding = model.encode([chunk.page_content])[0]

                    # Asignar el título y otros metadatos al documento
                    chunk_metadata = {
                        "title": files['title'].iloc[int(filename[:-4])],  # Asignamos el título a los metadatos
                        "file_name": filename,  # Puedes agregar otros metadatos si lo deseas
                        "chunk_index": j  # Si necesitas un índice de chunk (opcional)
                    }
                    print(chunk_metadata)
                    metadata.append(chunk_metadata)
                    ids.append("ID" + str(j))
                    j+=1
            except:
                print(files['title'].iloc[int(filename[:-4])])

{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 0}
{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 1}
{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 2}
{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 3}
{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 4}
{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 5}
{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 6}
{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 7}
{'title': 'Introducción al Desarrollo Web', 'file_name': '0.txt', 'chunk_index': 8}
{'title': 'Introducción al Análisis de datos con Python', 'file_name': '1.txt', 'chunk_index': 9}
{'title': 'Introducción al Análisis de datos con Python', 'file_name': '1.txt', 'chunk_index': 10}
{'title': 'Introducción al Análisis de datos co

In [5]:
#adding to chromadb
collection.upsert(
    documents = documents,
    metadatas = metadata,
    ids = ids
)